# Computed Formant Error 
computed absolute formant error of each model
- Created 1 April 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import librosa
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import librosa.display
from matplotlib import cm
from functools import partial

import sklearn
from sklearn.decomposition import PCA
from os import makedirs
import os
from os.path import join
import shutil
from IPython import display

%matplotlib inline

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# np.random.seed(42)

In [3]:
def numpy_fillna(data):
    lens = np.array([len(i) for i in data])
    mask = np.arange(lens.max()) < lens[:,None]
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out

### Monosyllablic vowel utterance 

In [6]:
EXP = [0,1,2,3,4,5,6]

In [7]:
res = []
for exp_num in EXP:
    path = '../experiment/result/eval_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))

    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))

    f1_error = np.absolute(np.mean(a1 - e1))
    f2_error = np.absolute(np.mean(a2 - e2))
    f3_error = np.absolute(np.mean(a3 - e3))
    res.append([exp_num, f1_error, f2_error, f3_error])
    
pd.DataFrame(np.array(res),columns=['MODEL', 'F1 ERROR','F2 ERROR', 'F3 ERROR'])

,MODEL,F1 ERROR,F2 ERROR,F3 ERROR
0,0.0,211.737148,457.963614,254.057941
1,1.0,8.154464,61.003424,142.749354
2,2.0,11.199595,13.304167,17.436882
3,3.0,13.289811,8.386883,5.561610
4,4.0,10.885531,7.968893,25.663965
5,5.0,4.920603,10.213605,27.826254
6,6.0,6.052730,17.685022,8.278641


### Disyllablic vowel utterance 

In [10]:
EXP = [7,8,9,10,11,12,13]

In [11]:
res = []
for exp_num in EXP:

    path = '../experiment/result/eval_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))
    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))
    
    
    
    e1 = numpy_fillna(e1)
    e2 = numpy_fillna(e2)
    e3 = numpy_fillna(e3)
    

    f1_error = a1 - e1
    f2_error = a2 - e2
    f3_error = a3 - e3

    
    f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
    f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
    f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))
    
    f11_error = np.absolute(np.mean(f1_error[:,0,:]))
    f21_error = np.absolute(np.mean(f1_error[:,1,:]))
    f12_error = np.absolute(np.mean(f2_error[:,0,:]))
    f22_error = np.absolute(np.mean(f2_error[:,1,:]))
    f13_error = np.absolute(np.mean(f3_error[:,0,:]))
    f23_error = np.absolute(np.mean(f3_error[:,1,:]))

    res.append([exp_num, f11_error, f12_error, f13_error, f21_error, f22_error, f23_error])

pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR', '2F1 ERROR','2F2 ERROR', '2F3 ERROR'])

,MODEL,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR
0,7.0,360.951068,552.420291,553.032562,226.853382,420.329982,404.198061
1,8.0,19.738270,113.669110,178.034972,19.276967,105.374893,160.039464
2,9.0,2.837395,13.415652,31.417564,3.989918,16.001064,40.747244
3,10.0,1.707653,30.237143,23.260197,4.377838,4.384458,13.915472
4,11.0,6.247161,12.811057,38.248174,4.140200,17.399088,34.212830
5,12.0,3.158708,0.203458,14.420970,0.774225,9.018479,5.250237
6,13.0,1.693502,23.288996,60.792563,3.035423,18.186394,47.768389


## Formant Error of each syllable 

### Monosyllablic vowel utterance 

In [34]:
exp_num = 141

In [35]:
path = '../experiment_2/result/eval_{}/formant/'.format(exp_num)

a1 = np.load(join(path, 'actual_F1.npy'))
a2 = np.load(join(path, 'actual_F2.npy'))
a3 = np.load(join(path, 'actual_F3.npy'))

e1 = np.load(join(path, 'estimated_F1.npy'))
e2 = np.load(join(path, 'estimated_F2.npy'))
e3 = np.load(join(path, 'estimated_F3.npy'))

f1_error = np.absolute(np.mean(a1 - e1, axis=1))
f2_error = np.absolute(np.mean(a2 - e2, axis=1))
f3_error = np.absolute(np.mean(a3 - e3, axis=1))


pd.DataFrame({ 'F1 ERROR': f1_error,'F2 ERROR':f2_error, 'F3 ERROR':f3_error},columns=['F1 ERROR','F2 ERROR', 'F3 ERROR'])

,F1 ERROR,F2 ERROR,F3 ERROR
0,9.193027,1.151415,61.032071
1,38.323501,23.791440,138.983825
2,24.689293,16.921870,18.038637
3,0.126191,9.073919,47.598814
4,26.398543,50.755641,22.982564
5,30.104164,142.992524,33.507221
6,10.561219,13.544982,31.527813
7,20.878645,25.024393,27.506083
8,24.503416,123.955822,102.259554
9,10.548538,29.694862,0.640413


### Disyllablic vowel utterance 

In [29]:
def change_label_set_1(datapoint_df, col):
    datapoint_df.at['E', col]= 'ɛ:' 
    datapoint_df.at['O', col]= 'ɔ:'
    datapoint_df.at['9', col]= 'œ:'
    datapoint_df.at['@', col]= 'ə:'
    datapoint_df.at['o', col]= 'o:'
    datapoint_df.at['a', col]= 'a:'
    datapoint_df.at['i', col]= 'i:'
    datapoint_df.at['e', col]= 'e:'
    datapoint_df.at['u', col]= 'u:'
    datapoint_df.at['A', col]= 'ɑ:'
    datapoint_df.at['2', col]= 'ø:'
    datapoint_df.at['U', col]= 'ʊ:'

    return datapoint_df

def custome_reindex_type2(df):
    return df.reindex(["a:", "i:", "u:","e:",'ɛ:','ə:','œ:','o:','ɔ:', 'ɑ:','ø:','ʊ:'])

def set_datapoint_index(df, col):
    df_temp = df.copy()
    df_temp['Label_idx'] = df_temp[col]
    return df_temp.set_index('Label_idx')

In [93]:
exp_num = 140

In [94]:
path = '../experiment/result/eval_{}/formant/'.format(exp_num)


with open('../data/d_eval/syllable_name.txt') as f:
    label = np.array([word.strip()[0]+';'+word.strip()[1] for line in f for word in line.split(',')])
    
a1 = np.load(join(path, 'actual_F1.npy'))
a2 = np.load(join(path, 'actual_F2.npy'))
a3 = np.load(join(path, 'actual_F3.npy'))

e1 = np.load(join(path, 'estimated_F1.npy'))
e2 = np.load(join(path, 'estimated_F2.npy'))
e3 = np.load(join(path, 'estimated_F3.npy'))

e1 = numpy_fillna(e1)
e2 = numpy_fillna(e2)
e3 = numpy_fillna(e3)

f1_error = a1 - e1
f2_error = a2 - e2
f3_error = a3 - e3


f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))

f11_error = np.absolute(np.mean(f1_error[:,0,:], axis=1))
f21_error = np.absolute(np.mean(f1_error[:,1,:], axis=1))
f12_error = np.absolute(np.mean(f2_error[:,0,:], axis=1))
f22_error = np.absolute(np.mean(f2_error[:,1,:], axis=1))
f13_error = np.absolute(np.mean(f3_error[:,0,:], axis=1))
f23_error = np.absolute(np.mean(f3_error[:,1,:], axis=1))
    
    
res_df = pd.DataFrame({ 'Pho':label, '1F1 ERROR': f11_error,'1F2 ERROR':f12_error, '1F3 ERROR':f13_error,
              '2F1 ERROR': f21_error,'2F2 ERROR':f22_error, '2F3 ERROR':f23_error})

res_df['Pho1'], res_df['Pho2'] = res_df['Pho'].str.split(';', 1).str

In [95]:
res_df.head()

,Pho,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR,Pho1,Pho2
0,a;i,3.721138,26.641260,49.638811,11.478654,20.129181,126.263750,a,i
1,a;u,4.298982,29.281510,50.904471,12.877228,47.955715,4.422733,a,u
2,a;e,4.302551,29.187249,51.037416,5.494803,3.468164,50.653916,a,e
3,a;E,3.740658,27.104133,50.545571,8.729425,28.549945,6.101305,a,E
4,a;o,4.254108,29.252248,51.086151,9.383158,34.026984,43.658125,a,o


In [96]:
res_1 = res_df.groupby(['Pho1']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_1(res_1, col='Pho1')
res_1 = set_datapoint_index(res_1, col='Pho1')
custome_reindex_type2(res_1).drop(['Pho1'], axis=1)

NameError: name 'change_label_set_1' is not defined

In [36]:
res_2 = res_df.groupby(['Pho2']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_1(res_2, col='Pho2')
res_2 = set_datapoint_index(res_2, col='Pho2')
custome_reindex_type2(res_2).drop(['Pho2'], axis=1)

,1F1 ERROR,1F2 ERROR,1F3 ERROR
Label_idx,,,
a:,13.635082,36.612971,40.965285
i:,11.146992,35.928388,24.943819
u:,12.421793,28.806141,43.113271
e:,12.694796,41.434180,44.453905
ɛ:,12.909823,36.603886,43.193302
ə:,11.916951,38.846014,43.410871
œ:,11.623216,37.032053,42.062268
o:,13.156882,31.715534,43.788306
ɔ:,12.709002,34.377758,42.457960


# Predict

In [10]:
def change_label_set_2(datapoint_df, col):
    datapoint_df.at['E', col]= 'ɛ:' 
    datapoint_df.at['O', col]= 'ɔ:'
    datapoint_df.at['7', col]= 'ɤ:'
    datapoint_df.at['M', col]= 'ɯ:'
    datapoint_df.at['o', col]= 'o:'
    datapoint_df.at['a', col]= 'a:'
    datapoint_df.at['i', col]= 'i:'
    datapoint_df.at['e', col]= 'e:'
    datapoint_df.at['u', col]= 'u:'
    return datapoint_df

def custome_reindex_type3(df):
    return df.reindex(["a:", "i:", "u:","e:",'ɛ:','ɯ:','ɤ:','o:','ɔ:'])

def set_datapoint_index(df, col):
    df_temp = df.copy()
    df_temp['Label_idx'] = df_temp[col]
    return df_temp.set_index('Label_idx')

In [9]:
EXP = ['145_1', '144_1','149','140_8', '143_1', '125','126','127_1','128']

In [11]:
exp_num = '10_12'

In [12]:
path = '../experiment/result/predict_{}/formant/'.format(exp_num)


with open('../data/d_records/selected/syllable_name.txt') as f:
    label = np.array([word.strip()[0]+';'+word.strip()[1] for line in f for word in line.split(',')])
    
a1 = np.load(join(path, 'actual_F1.npy'))
a2 = np.load(join(path, 'actual_F2.npy'))
a3 = np.load(join(path, 'actual_F3.npy'))

e1 = np.load(join(path, 'estimated_F1.npy'))
e2 = np.load(join(path, 'estimated_F2.npy'))
e3 = np.load(join(path, 'estimated_F3.npy'))

a1 = numpy_fillna(a1)
a2 = numpy_fillna(a2)
a3 = numpy_fillna(a3)

e1 = numpy_fillna(e1)
e2 = numpy_fillna(e2)
e3 = numpy_fillna(e3)

f1_error = np.absolute(a1 - e1[:,1:])
f2_error = np.absolute(a2 - e2[:,1:])
f3_error = np.absolute(a3 - e3[:,1:])

f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))

f11_error = np.mean(f1_error[:,0,:], axis=1).astype('float')
f21_error = np.mean(f1_error[:,1,:], axis=1).astype('float')
f12_error = np.mean(f2_error[:,0,:], axis=1).astype('float')
f22_error = np.mean(f2_error[:,1,:], axis=1).astype('float')
f13_error = np.mean(f3_error[:,0,:], axis=1).astype('float')
f23_error = np.mean(f3_error[:,1,:], axis=1).astype('float')

    
res_df = pd.DataFrame({ 'Pho':label, '1F1 ERROR': f11_error,'1F2 ERROR':f12_error, '1F3 ERROR':f13_error,
              '2F1 ERROR': f21_error,'2F2 ERROR':f22_error, '2F3 ERROR':f23_error})

res_df['Pho1'], res_df['Pho2'] = res_df['Pho'].str.split(';', 1).str

In [13]:
res_df.describe()

,1F1 ERROR,1F2 ERROR,1F3 ERROR,2F1 ERROR,2F2 ERROR,2F3 ERROR
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,49.413064,237.889485,353.187015,138.749783,399.788425,494.239196
std,21.008131,135.038978,124.252309,117.593058,228.620230,224.659507
min,13.505584,61.294599,100.517629,29.937316,134.758289,151.303932
25%,34.067368,148.265694,262.358842,82.209058,306.196751,380.628358
50%,47.361265,221.761136,343.202632,117.856334,353.457480,456.681784
75%,65.127532,304.108502,442.987847,165.135968,432.157856,572.546744
max,113.325946,928.577286,652.716035,849.036229,1378.045392,1808.346210


In [14]:
res_1 = res_df.groupby(['Pho1']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_2(res_1, col='Pho1')
res_1 = set_datapoint_index(res_1, col='Pho1')
custome_reindex_type3(res_1).drop(['Pho1'], axis=1)

,1F1 ERROR,1F2 ERROR,1F3 ERROR
Label_idx,,,
a:,72.198761,207.057727,243.647668
i:,31.496482,223.099176,538.839757
u:,38.396171,335.088807,381.351962
e:,38.175494,344.998606,360.393330
ɛ:,47.993416,333.138719,237.091676
ɯ:,41.934275,167.732034,319.025560
ɤ:,51.543618,175.582307,322.532503
o:,63.670448,165.673136,438.188169
ɔ:,59.308909,188.634856,337.612514


In [15]:
res_2 = res_df.groupby(['Pho2']).mean()[['1F1 ERROR', '1F2 ERROR','1F3 ERROR']]

change_label_set_2(res_2, col='Pho2')
res_2 = set_datapoint_index(res_2, col='Pho2')
custome_reindex_type3(res_2).drop(['Pho2'], axis=1)

,1F1 ERROR,1F2 ERROR,1F3 ERROR
Label_idx,,,
a:,48.232559,262.876347,368.320563
i:,53.295149,292.552603,417.514420
u:,51.735336,312.456986,375.954902
e:,52.830667,272.878134,397.797639
ɛ:,55.446328,220.649361,314.465623
ɯ:,49.481068,229.552341,309.844023
ɤ:,38.503404,207.696849,312.038399
o:,48.067960,190.149863,379.133043
ɔ:,47.125104,152.192885,303.614525


# Compare Model (Predict)

### Disyllablic vowel utterance 

In [87]:
EXP = ['145_1', '144_1','149','140_8', '143_1', '125','126','127_1','128']

In [90]:
res = []
for exp_num in EXP:

    path = '../experiment_2/result/predict_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))
    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))
    
    a1 = numpy_fillna(a1)
    a2 = numpy_fillna(a2)
    a3 = numpy_fillna(a3)
    
    e1 = numpy_fillna(e1)
    e2 = numpy_fillna(e2)
    e3 = numpy_fillna(e3)
    
#     print(a1[0])
#     print(e1[0,1:])
#     print(a1 - e1[:,:a1.shape[1]])
    

    f1_error = np.absolute(a1 - e1[:,1:])
    f2_error = np.absolute(a2 - e2[:,1:])
    f3_error = np.absolute(a3 - e3[:,1:])

    
#     f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
#     f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
#     f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))
    
#     f11_error = np.mean(f1_error[:,0,:])
#     f21_error = np.mean(f1_error[:,1,:])
#     f12_error = np.mean(f2_error[:,0,:])
#     f22_error = np.mean(f2_error[:,1,:])
#     f13_error = np.mean(f3_error[:,0,:])
#     f23_error = np.mean(f3_error[:,1,:])

#     res.append([exp_num, f11_error, f12_error, f13_error, f21_error, f22_error, f23_error])

# pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR', '2F1 ERROR','2F2 ERROR', '2F3 ERROR'])

    f11_error = np.mean(f1_error)
    f12_error = np.mean(f2_error)
    f13_error = np.mean(f3_error)
    res.append([exp_num, f11_error, f12_error, f13_error])

pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR'])

,MODEL,1F1 ERROR,1F2 ERROR,1F3 ERROR
0,145_1,112.00416820825957,217.09364589836733,350.45319714573657
1,144_1,119.95830731732016,236.89677923119248,396.31107404686617
2,149,88.48436175338955,235.07249469273603,364.80895702479756
3,140_8,92.83169765752909,224.8607146672729,343.951202184042
4,143_1,88.77901339018256,240.690887553348,351.86684620219444
5,125,244.29926396559785,403.96585486955655,523.0060945120985
6,126,93.39458647676918,248.1208477313751,370.04322457604815
7,127_1,91.6241436019736,237.63273881403268,340.8295699924575
8,128,93.16484865021951,236.52852811808498,375.77205858331894


In [89]:
res = []
for exp_num in EXP:

    path = '../experiment_2/result/predict_{}/formant/'.format(exp_num)

    a1 = np.load(join(path, 'actual_F1.npy'))
    a2 = np.load(join(path, 'actual_F2.npy'))
    a3 = np.load(join(path, 'actual_F3.npy'))
    e1 = np.load(join(path, 'estimated_F1.npy'))
    e2 = np.load(join(path, 'estimated_F2.npy'))
    e3 = np.load(join(path, 'estimated_F3.npy'))
    
    a1 = numpy_fillna(a1)
    a2 = numpy_fillna(a2)
    a3 = numpy_fillna(a3)
    
    e1 = numpy_fillna(e1)
    e2 = numpy_fillna(e2)
    e3 = numpy_fillna(e3)
    

    f1_error = a1 - e1[:,1:]
    f2_error = a2 - e2[:,1:]
    f3_error = a3 - e3[:,1:]

    
#     f1_error = f1_error.reshape((f1_error.shape[0]//2,2,f1_error.shape[1]))
#     f2_error = f2_error.reshape((f2_error.shape[0]//2,2,f2_error.shape[1]))
#     f3_error = f3_error.reshape((f3_error.shape[0]//2,2,f3_error.shape[1]))
    
#     f11_error = np.std(f1_error[:,0,:])
#     f21_error = np.std(f1_error[:,1,:])
#     f12_error = np.std(f2_error[:,0,:])
#     f22_error = np.std(f2_error[:,1,:])
#     f13_error = np.std(f3_error[:,0,:])
#     f23_error = np.std(f3_error[:,1,:])

    f11_error = np.std(f1_error)
    f12_error = np.std(f2_error)
    f13_error = np.std(f3_error)

#     res.append([exp_num, f11_error, f12_error, f13_error, f21_error, f22_error, f23_error])
    res.append([exp_num, f11_error, f12_error, f13_error])

# pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR', '2F1 ERROR','2F2 ERROR', '2F3 ERROR'])
pd.DataFrame(np.array(res),columns=['MODEL', '1F1 ERROR','1F2 ERROR', '1F3 ERROR'])

,MODEL,1F1 ERROR,1F2 ERROR,1F3 ERROR
0,145_1,112.63166662902886,357.88906654595394,373.9314513757186
1,144_1,175.02366170492377,399.8194549537851,475.82294659649307
2,149,100.94761128382662,368.4575425437563,354.7726493069169
3,140_8,97.16321811567431,341.3264341242244,322.4220617840579
4,143_1,104.01637878459249,364.2676907230029,346.139583573839
5,125,421.64273005773174,660.1778239034861,886.801265905272
6,126,101.58323186850524,355.37709242428724,346.1072502128624
7,127_1,97.6679990108144,342.30268906870396,332.38360574356915
8,128,99.80314541007772,337.6671241982641,335.26480943493226


In [11]:
f1_error[:,0,:]

array([[-322.73438464504864, -72.48963865805672, 232.115639740554, ...,
        0, 0, 0],
       [-145.44469633072595, 494.328074980915, 357.7102471071097, ...,
        -677.4121722077449, 0, 0],
       [-245.2587294605247, 361.97275230528084, 201.89266691817295, ...,
        0, 0, 0],
       ...,
       [-885.3538404679518, -558.6593773510252, 108.55343484066577, ...,
        0, 0, 0],
       [-1526.9052579402062, 298.51467352435986, 147.00312709428795, ...,
        0, 0, 0],
       [-253.25402443291273, -69.50859609316683, 256.171453316303, ...,
        0, 0, 0]], dtype=object)

In [12]:
f1_error

array([[[-322.73438464504864, -72.48963865805672, 232.115639740554, ...,
         0, 0, 0],
        [-85.54668650649432, -113.10154910828726, -133.5090548288349,
         ..., 0, 0, 0]],

       [[-145.44469633072595, 494.328074980915, 357.7102471071097, ...,
         -677.4121722077449, 0, 0],
        [-102.79981643592157, -136.61865814199837, -139.16437188073837,
         ..., -150.02769338968773, 0, 0]],

       [[-245.2587294605247, 361.97275230528084, 201.89266691817295,
         ..., 0, 0, 0],
        [125.44044468098195, 142.96496350485768, 129.00131097663882,
         ..., 0, 0, 0]],

       ...,

       [[-885.3538404679518, -558.6593773510252, 108.55343484066577,
         ..., 0, 0, 0],
        [44.46079280247977, 45.95138342716439, 45.68595583977617, ...,
         0, 0, 0]],

       [[-1526.9052579402062, 298.51467352435986, 147.00312709428795,
         ..., 0, 0, 0],
        [115.86032479957515, 121.69313254646232, 111.41474624492042,
         ..., 0, 0, 0]],

       [[-253

In [25]:
for i in range(1):
    print(1)

1
